In [1]:
import numpyro
import numpyro.distributions as dist
import numpy as np
import arviz as az
from jax.random import PRNGKey
from numpyro.distributions.transforms import AffineTransform
from numpyro.infer import MCMC, NUTS, Predictive
numpyro.set_platform("cpu")

eight_school_data = {
    "J": 8,
    "y": np.array([28.0, 8.0, -3.0, 7.0, -1.0, 1.0, 18.0, 12.0]),
    "sigma": np.array([15.0, 10.0, 16.0, 11.0, 9.0, 11.0, 10.0, 18.0]),
}


def model(J, sigma, y=None):
    mu = numpyro.sample("mu", dist.Normal(0, 5))
    tau = numpyro.sample("tau", dist.HalfCauchy(5))
    # use non-centered reparameterization
    theta = numpyro.sample(
        "theta",
        dist.TransformedDistribution(dist.Normal(np.zeros(J), 1), AffineTransform(mu, tau)),
    )
    numpyro.sample("y", dist.Normal(theta, sigma), obs=y)


kernel = NUTS(model)
mcmc = MCMC(kernel, num_warmup=500, num_samples=500, num_chains=4)
mcmc.run(PRNGKey(0), **eight_school_data, extra_fields=["num_steps", "energy"])
posterior_samples = mcmc.get_samples()
posterior_predictive = Predictive(model, posterior_samples)(
    PRNGKey(1), eight_school_data["J"], eight_school_data["sigma"]
)
prior = Predictive(model, num_samples=500)(
    PRNGKey(2), eight_school_data["J"], eight_school_data["sigma"]
)

numpyro_data = az.from_numpyro(
    mcmc,
    prior=prior,
    posterior_predictive=posterior_predictive,
    coords={"school": np.arange(eight_school_data["J"])},
    dims={"theta": ["school"]},
)
numpyro_data

/Users/heningwang/Documents/GitHub/numpyro_adjective_modelling/jax-metal/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Platform 'METAL' is experimental and not all JAX functionality may be correctly supported!
2025-11-28 14:21:07.919511: W pjrt_plugin/src/mps_client.cc:563] WARNING: JAX Apple GPU support is experimental and not all JAX functionality is correctly supported!
/var/folders/jc/6ctllcx143xbrgx01x69skg80000gn/T/ipykernel_67513/3307295101.py:29: UserWarning: There are not enough devices to run parallel chains: expected 4 but got 1. Chains will be drawn sequentially. If you are running MCMC in CPU, consider using `numpyro.set_host_device_count(4)` at the beginning of your program. You can double-check how many devices are available in your system using `jax.local_device_count()`.
  mcmc = MCMC

Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.92 GB



sample: 100%|██████████| 1000/1000 [00:00<00:00, 8360.72it/s, 31 steps of size 2.24e-01. acc. prob=0.80]


Inference data with groups:
	> posterior
	> posterior_predictive
	> log_likelihood
	> sample_stats
	> prior
	> prior_predictive
	> observed_data

In [5]:
numpyro_data.prior_predictive.y

<xarray.DataArray 'y' (chain: 1, draw: 500, y_dim_0: 8)> Size: 16kB
array([[[ 19.69039   ,  -0.34815684, -22.367302  , ...,  16.622828  ,
         -15.7688055 ,  -6.845168  ],
        [  9.477537  ,  -9.470676  ,   8.480471  , ...,  -3.1395402 ,
           8.926897  ,  21.148172  ],
        [-12.141536  ,  20.385244  ,  14.313737  , ...,  -8.920131  ,
          -8.540659  ,  10.570226  ],
        ...,
        [  6.195146  , -16.81244   , -21.583776  , ...,  -6.104437  ,
         -19.76478   , -15.566423  ],
        [ 23.94809   , -19.047075  ,  -6.228316  , ...,   9.32434   ,
          15.134216  ,   0.15730613],
        [ 18.839058  ,  -3.9360518 , -30.911882  , ..., -25.759172  ,
          44.76303   , -34.69204   ]]], dtype=float32)
Coordinates:
  * chain    (chain) int64 8B 0
  * draw     (draw) int64 4kB 0 1 2 3 4 5 6 7 ... 493 494 495 496 497 498 499
  * y_dim_0  (y_dim_0) int64 64B 0 1 2 3 4 5 6 7